In [ ]:
import jax
import jax.numpy as jnp
import sys
from functools import map
sys.path.append("../..")

In [2]:
z = jnp.zeros((2, 3, 5, 6))
z2 = jnp.zeros((2, 3, 5, 6))


2024-11-18 21:32:06.768103: W external/xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version 12.6.77. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [ ]:
map(jnp.moveaxis

In [2]:
jnp.dtype

numpy.dtype

In [5]:
arr = jnp.arange(5)[None, None, :].astype(jnp.float32)
arr.shape, arr

((1, 1, 5), Array([[[0., 1., 2., 3., 4.]]], dtype=float32))

In [8]:
arr.dtype == jnp.float32, arr.dtype==jnp.bfloat16

(True, False)

In [4]:
arr_rep = arr[:, :, :, None].repeat(3, axis=-1)
arr_rep.shape, arr_rep

((1, 1, 5, 3),
 Array([[[[0, 0, 0],
          [1, 1, 1],
          [2, 2, 2],
          [3, 3, 3],
          [4, 4, 4]]]], dtype=int32))

In [5]:
import flax.linen as nn

import jax

import numpy as np

x = jax.random.normal(jax.random.key(0), (3, 4, 5, 6))

layer = nn.LayerNorm()

variables = layer.init(jax.random.key(1), x)
variables

{'params': {'scale': Array([1., 1., 1., 1., 1., 1.], dtype=float32),
  'bias': Array([0., 0., 0., 0., 0., 0.], dtype=float32)}}

In [6]:
y = layer.apply(variables, x)
y.mean(-1), y.std(-1)

(Array([[[-1.98682155e-08,  3.97364310e-08,  1.19209290e-07,
          -1.98682155e-08,  3.97364310e-08],
         [-4.22199591e-08,  1.49011612e-08,  3.97364310e-08,
          -9.93410776e-09, -2.48352694e-09],
         [ 0.00000000e+00, -1.98682155e-08, -1.09275184e-07,
          -4.34617213e-08,  9.93410776e-09],
         [ 1.24176349e-08,  1.98682155e-08,  9.93410776e-09,
          -1.98682155e-08, -9.93410776e-09]],
 
        [[-3.97364310e-08,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  3.97364310e-08],
         [ 1.49011612e-08, -9.93410776e-09,  1.98682155e-08,
          -1.65154546e-07, -1.98682155e-08],
         [ 2.23517418e-08, -4.96705388e-09,  0.00000000e+00,
           9.93410776e-09, -2.98023224e-08],
         [ 1.98682155e-08,  3.97364310e-08, -2.98023224e-08,
           3.47693785e-08, -2.98023224e-08]],
 
        [[ 1.98682155e-08,  1.98682155e-08, -3.60887498e-08,
           1.49011612e-08,  3.97364310e-08],
         [ 1.68879836e-07, -1.98682155e-

In [7]:

# y = nn.LayerNorm(reduction_axes=(1, 2, 3)).apply(variables, x)

# y2 = nn.GroupNorm(num_groups=1).apply(variables, x)

# np.testing.assert_allclose(y, y2)

# y = nn.LayerNorm(reduction_axes=(1, 2), feature_axes=-1).apply(variables, x)

# y2 = nn.InstanceNorm(feature_axes=-1).apply(variables, x)

# np.testing.assert_allclose(y, y2)

In [8]:
from mlstm_kernels.mlstm_kernels.jax.components.normalization import MultiHeadNormLayer

In [9]:
norm_layer = MultiHeadNormLayer(weight=True, bias=False, eps=1e-5, dtype=jnp.float32, axis=1, norm_type="layernorm")

In [12]:
variables = norm_layer.init(jax.random.PRNGKey(0), x)
type(variables), variables

(dict,
 {'params': {'scale': Array([[1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1.]], dtype=float32)}})

In [11]:
norm_layer.apply(variables, x)

Array([[[[-1.4092555 , -0.02534706,  1.3489714 , -0.59527695,
           1.236951  , -0.55604297],
         [-0.97789   , -1.474389  ,  0.97012866, -0.1211729 ,
           0.24100842,  1.3623149 ],
         [-1.3442954 ,  1.3677758 ,  0.17286192, -1.1040248 ,
          -0.12115045,  1.0288335 ],
         [-0.8313368 ,  0.21111865,  1.7076395 , -0.90581435,
          -0.9600632 ,  0.77845615],
         [-0.23060739, -0.4206881 , -0.36171237, -0.46290994,
          -0.734365  ,  2.210283  ]],

        [[-2.0176628 , -0.06930881,  0.73434585,  0.44363692,
           1.0769527 , -0.16796398],
         [ 1.5571115 , -1.3184897 ,  1.1080704 , -0.42524126,
          -0.5039902 , -0.4174607 ],
         [-0.13483013,  0.90799266, -0.5444893 , -1.3137347 ,
          -0.5857985 ,  1.6708603 ],
         [-1.3847338 , -0.6320176 ,  1.6170238 ,  0.47449353,
          -0.6855187 ,  0.6107526 ],
         [ 1.8700753 , -0.2642717 , -1.396383  , -0.6285463 ,
           0.20935799,  0.20976764]],

      

In [16]:
from flax import nnx
import flax
flax.__version__

'0.9.0'

In [19]:
class WeightStack(nnx.Module):
  @nnx.vmap(in_axes=(0, 0), out_axes=0)
  def __init__(self, seed: jax.Array):
    self.kernel = nnx.Param(jax.random.uniform(jax.random.key(seed), (2, 3)))
    self.bias = nnx.Param(jnp.zeros((3,)))

  @nnx.vmap(in_axes=(0, 0), out_axes=1)
  def __call__(self, x: jax.Array):
    assert len(self.kernel.shape) == 2, 'Batch dimensions not allowed'
    assert x.ndim == 1, 'Batch dimensions not allowed'
    return x @ self.kernel + self.bias

weights = WeightStack(jnp.arange(10))

x = jax.random.normal(jax.random.key(1), (10, 2))
y = weights(x)


In [20]:
weights

WeightStack(
  bias=Param(
    value=Array(shape=(10, 3), dtype=float32)
  ),
  kernel=Param(
    value=Array(shape=(10, 2, 3), dtype=float32)
  )
)